# Property Manager Municipal Remittance Data

### Step 1:

Extracted 1.5 GB of municipal real estate property manager financial information from an XLSX formatted file. 

Transformed the data by joining data across the 14 worksheets, removing duplicates and nulls, and grouping by Owner_Name.

(Grouping is not entirely necessary at this point since multiple dimensions exist. The data will be focused, and unnecessary dimensions will be removed in step 2. However, this highlights the issue for discussion.)

Loaded a new file from the cleaned data to reduce its size, extract, and better transform the data in step 2.

In [6]:
import pandas as pd

# Load the Excel file
file_path = '7.14.24 UPIL.xlsx'
xls = pd.ExcelFile(file_path)

# Read all sheets into a dictionary of DataFrames
dfs = {}
for sheet_name in xls.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0 if sheet_name == 'Sheet1' else None)
    # Store the DataFrame in the dictionary
    dfs[sheet_name] = df

# Use the headers from 'Sheet1' for all DataFrames
headers = dfs['Sheet1'].columns
for sheet_name, df in dfs.items():
    if sheet_name != 'Sheet1':
        df.columns = headers

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs.values(), ignore_index=True)

# Clean 'Owner_Name' column
combined_df['Owner_Name'] = combined_df['Owner_Name'].str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)

# Remove duplicates and nulls, then group by 'Owner_Name'
cleaned_df = combined_df.drop_duplicates().dropna().groupby('Owner_Name').apply(lambda x: x) #needs agg

# Write the cleaned DataFrame to a new Excel file
cleaned_df.reset_index(drop=True).to_excel('cleaned_workbook.xlsx', index=False)

### Step 2:

Cleaned raw data, consulting for next steps and finer points

In [7]:
import pandas as pd

# Import the cleaned workbook
cleaned_file_path = 'cleaned_workbook.xlsx'

# Load the cleaned workbook
df = pd.read_excel(cleaned_file_path)

# Clean 'Owner_Name' column again to ensure consistency
df['Owner_Name'] = df['Owner_Name'].str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)

# Define aggregation functions
# Assuming the value to sum is in column 'Value' and other columns should keep their first occurrence
agg_funcs = {
    'Cash_Remitted' 'OwnerCount' 'Shares': 'sum',  # Replace 'Value' with the actual column name you want to sum
    'Other_Info': 'first'  # Replace 'Other_Info' with actual non-numeric column names
}

# Group by 'Owner_Name' and aggregate
grouped_df = df.groupby('Owner_Name').agg(agg_funcs).reset_index()

# Write the grouped DataFrame to a new Excel file
grouped_df.to_excel('grouped_cleaned_workbook.xlsx', index=False)